## Extracting embeddings from ESM C

In [6]:
import torch
from Bio import SeqIO
from esm.models.esmc import ESMC


In [ ]:
#pip install --no-cache-dir esm

In [ ]:
from huggingface_hub import interpreter_login, whoami

interpreter_login()

print(whoami()) 

/stor/work/Wilke/luiz/ESM3/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

{'type': 'user', 'id': '6533f78325a58aec658be329', 'name': 'ziulbio', 'fullname': 'Luiz Vieira', 'email': 'ziul.bio@gmail.com', 'emailVerified': True, 'canPay': False, 'periodEnd': None, 'isPro': False, 'avatarUrl': '/avatars/ccbd3b6a2d8350896fad577104aad2e5.svg', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'ESM3_HF_token', 'role': 'read', 'createdAt': '2024-12-06T02:54:05.173Z'}}}


# This new version of the esm, download the model at:   
~/.cache/huggingface/hub/models--EvolutionaryScale--esmc-300m-2024-12/snapshots/15e3c02ec986679ecddd449c02c7186cf554b985/data/weights/esmc_300m_2024_12_v0.pth

In [ ]:
from esm.models.esmc import ESMC
from esm.sdk.api import ESMProtein, LogitsConfig

protein = ESMProtein(sequence="AAAAA")
client = ESMC.from_pretrained("esmc_300m").to("cuda") # or "cpu"
protein_tensor = client.encode(protein)
logits_output = client.logits(
   protein_tensor, LogitsConfig(sequence=True, return_embeddings=True)
)

In [32]:
print(logits_output.embeddings)

tensor([[[ 0.0094, -0.0085,  0.0038,  ...,  0.0176,  0.0098, -0.0062],
         [ 0.0050, -0.0243,  0.0280,  ...,  0.0576,  0.0276,  0.0108],
         [ 0.0131, -0.0457, -0.0076,  ...,  0.0457,  0.0125, -0.0082],
         ...,
         [ 0.0090, -0.0459, -0.0151,  ...,  0.0347,  0.0060,  0.0022],
         [ 0.0216, -0.0347, -0.0197,  ...,  0.0164,  0.0007, -0.0042],
         [-0.0023, -0.0206, -0.0201,  ..., -0.0043, -0.0012, -0.0182]]],
       dtype=torch.bfloat16)


In [ ]:
print(logits_output.embeddings)

tensor([[[ 0.0093, -0.0086,  0.0034,  ...,  0.0174,  0.0097, -0.0062],
         [ 0.0047, -0.0249,  0.0272,  ...,  0.0577,  0.0275,  0.0107],
         [ 0.0133, -0.0461, -0.0076,  ...,  0.0459,  0.0122, -0.0082],
         ...,
         [ 0.0088, -0.0461, -0.0151,  ...,  0.0346,  0.0058,  0.0025],
         [ 0.0217, -0.0349, -0.0197,  ...,  0.0162,  0.0003, -0.0045],
         [-0.0025, -0.0209, -0.0202,  ..., -0.0040, -0.0012, -0.0183]]],
       device='cuda:0')


# Example usage: Directly using the model

In [ ]:
# two line of code are necessery to change to garantee reprocucibility

#in file esmc.py at: venv/lib/python3.10/site-packages/esm/models/esmc.py


# line 66, change torch.bfloat16 to torch.float32
# if device.type != "cpu":
#        model = model.to(torch.bfloat16)


#and 

# change autocast to False and dtype to float32 at line 153:
# torch.autocast(enabled=True, device_type=device.type, dtype=torch.bfloat16)  # type: ignore


In [2]:
from esm.models.esmc import ESMC
model = ESMC.from_pretrained("esmc_300m").to("cuda") # "cuda" or "cpu"
print("Device:", model.device)
model

/stor/work/Wilke/luiz/SWAT/venv3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 3 files: 100%|██████████| 3/3 [00:00<00:00, 1434.60it/s]
/stor/work/Wilke/luiz/SWAT/venv3.10/lib/python3.10/site-packages/esm/pretrained.py:70: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless 

Device: cuda:0


ESMC(
  (embed): Embedding(64, 960)
  (transformer): TransformerStack(
    (blocks): ModuleList(
      (0-29): 30 x UnifiedTransformerBlock(
        (attn): MultiHeadAttention(
          (layernorm_qkv): Sequential(
            (0): LayerNorm((960,), eps=1e-05, elementwise_affine=True)
            (1): Linear(in_features=960, out_features=2880, bias=False)
          )
          (out_proj): Linear(in_features=960, out_features=960, bias=False)
          (q_ln): LayerNorm((960,), eps=1e-05, elementwise_affine=True)
          (k_ln): LayerNorm((960,), eps=1e-05, elementwise_affine=True)
          (rotary): RotaryEmbedding()
        )
        (ffn): Sequential(
          (0): LayerNorm((960,), eps=1e-05, elementwise_affine=True)
          (1): Linear(in_features=960, out_features=5120, bias=False)
          (2): SwiGLU()
          (3): Linear(in_features=2560, out_features=960, bias=False)
        )
      )
    )
    (norm): LayerNorm((960,), eps=1e-05, elementwise_affine=True)
  )
  (sequ

In [1]:
sequences = ["AAAAAA"*600]
sequences

['AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA

In [3]:
def extract(model, sequences):
    input_ids = model._tokenize(sequences)
    output = model(input_ids)
    logits, embeddings, hiddens = (
        output.sequence_logits,
        output.embeddings,
        output.hidden_states,
    )
    return logits, embeddings, hiddens

In [4]:
# One sequence
logits, embeddings, hiddens = extract(model, sequences)

In [5]:
embeddings.shape

torch.Size([1, 3602, 960])

In [6]:
embeddings[0,1:-1,:].detach().cpu().squeeze(0).shape

torch.Size([3600, 960])

In [7]:
print(hiddens.shape)
hiddens[-1][0][0][:10]

torch.Size([30, 1, 3602, 960])


tensor([ 39.6361, -48.3305, -69.6139, -84.0833, -25.2429, -35.1673, -59.3847,
          7.0865,  44.7230,  11.1606], device='cuda:0',
       grad_fn=<SliceBackward0>)

In [12]:
representations = hiddens[29]
representations.shape

torch.Size([1, 288, 960])

In [13]:
print(representations[0].shape)
representations[0]

torch.Size([288, 960])


tensor([[  22.4469,   81.8046,    0.4214,  ...,   36.7447,  -15.8932,
         -114.3045],
        [ -55.9338,  161.5620,  140.0640,  ...,   56.8568,    2.7359,
           83.2794],
        [ -37.2179,   69.2564,   11.3517,  ...,   62.3126,    9.4052,
           48.8188],
        ...,
        [-156.8451,  -22.2522,   54.4347,  ..., -111.8408, -235.4225,
            8.4382],
        [ -47.9209,  114.2353,   14.8654,  ...,   37.0107, -156.7603,
          -67.2377],
        [  35.5107,   59.9253,  -14.0217,  ...,   -9.3798,  -92.1953,
         -161.3322]], device='cuda:0', grad_fn=<SelectBackward0>)

In [72]:
import torch
torch.load('test/embeddings/BLAT_Ten_H24L.pt', weights_only=True).shape

torch.Size([286, 2560])

In [ ]:
# Many sequences
sequences = ["AA", "AAA", "AAAAA"]

In [17]:
sequences = [str(seq.seq) for seq in SeqIO.parse("test.fasta", "fasta")]
sequences

['MSIQHFRVALIPFFAAFCLPVFAPPETLVKVKDAEDQLGARVGYIELDLNSGKILESFRPEERFPMMSTFKVLLCGAVLSRIDAGQEQLGRRIHYSQNDLVEYSPVTEKHLTDGMTVRELCSAAITMSDNTAANLLLTTIGGPKELTAFLHNMGDHVTRLDRWEPELNEAIPNDERDTTMPVAMATTLRKLLTGELLTLASRQQLIDWMEADKVAGPLLRSALPAGWFIADKSGAGERGSRGIIAALGPDGKPSRIVVIYTTGSQATMDERNRQIAEIGASLIKHW',
 'MSIQHFRVALIPFFAAFCLPVFARPETLVKVKDAEDQLGARVGYIELDLNSGKILESFRPEERFPMMSTFKVLLCGAVLSRIDAGQEQLGRRIHYSQNDLVEYSPVTEKHLTDGMTVRELCSAAITMSDNTAANLLLTTIGGPKELTAFLHNMGDHVTRLDRWEPELNEAIPNDERDTTMPVAMATTLRKLLTGELLTLASRQQLIDWMEADKVAGPLLRSALPAGWFIADKSGAGERGSRGIIAALGPDGKPSRIVVIYTTGSQATMDERNRQIAEIGASLIKHW',
 'MSIQHFRVALIPFFAAFCLPVFAYPETLVKVKDAEDQLGARVGYIELDLNSGKILESFRPEERFPMMSTFKVLLCGAVLSRIDAGQEQLGRRIHYSQNDLVEYSPVTEKHLTDGMTVRELCSAAITMSDNTAANLLLTTIGGPKELTAFLHNMGDHVTRLDRWEPELNEAIPNDERDTTMPVAMATTLRKLLTGELLTLASRQQLIDWMEADKVAGPLLRSALPAGWFIADKSGAGERGSRGIIAALGPDGKPSRIVVIYTTGSQATMDERNRQIAEIGASLIKHW',
 'MSIQHFRVALIPFFAAFCLPVFALPETLVKVKDAEDQLGARVGYIELDLNSGKILESFRPEERFPMMSTFKVLLCGAVLSRIDAGQEQLGRRIHYSQNDLVEYSPVTEKHLTDGMTVRELCSAAI

In [18]:
logits, embeddings, hiddens = extract(model, sequences)

In [19]:
hiddens.shape

torch.Size([30, 7, 288, 960])

In [ ]:
# last layer
representations = hiddens[-1]
representations.shape

torch.Size([7, 288, 960])

In [22]:
# first sequence BOS token
representations[0][0]

tensor([ 2.6504e+01,  8.5209e+01,  2.7654e+00, -6.1944e+01, -4.8077e+01,
        -4.0412e+01, -5.4907e+01,  1.1671e+02,  6.4442e+00, -5.0825e+01,
        -3.2318e+01, -8.3447e+01,  1.4243e+02,  4.2467e+01, -2.7624e+02,
        -5.6971e+01, -9.8260e+01, -7.2435e+01, -1.2580e+02, -1.2523e+02,
         9.6887e+01, -9.6029e+01,  1.5023e+01,  6.4379e+02,  1.4658e+02,
        -2.5259e+01, -9.3657e+01, -2.7635e+01, -2.2957e+01,  7.2403e+01,
         2.0647e+00, -4.8618e+01,  1.2638e+02,  1.0582e+02, -2.1423e+01,
        -2.2720e+01,  8.5482e+00,  5.0017e+00, -6.5632e+00,  6.9960e+01,
        -3.3360e+01,  8.8610e+00,  6.0461e+01, -3.5431e+01, -3.7951e+01,
        -6.9472e+01,  8.0100e+00,  4.7792e+00,  1.4238e+01,  2.2161e+01,
         4.2488e+00,  2.4308e+01,  1.0211e+01,  2.5086e+02,  3.2126e+01,
         6.0596e+01,  8.6257e+01,  1.8820e+01, -1.1925e+01,  2.4125e+01,
        -1.8647e+00, -3.6488e+01, -3.9737e+00,  1.7308e+01,  1.0654e+02,
         1.3744e+01,  6.1214e+01,  2.8856e+01,  8.5

In [24]:
mean_representations= {}

for i, seq in enumerate(sequences):
    mean_representations[i] = representations[i, 1:len(seq)+1, :].mean(dim=0).detach().cpu()

mean_representations[0]

tensor([ 2.3120e+01,  2.1587e+01,  1.2867e+01, -3.7298e+01,  5.4642e+00,
        -4.0836e+01, -5.7978e+00,  1.6621e+01, -3.8429e+01,  1.5015e+01,
         6.9223e+01, -2.3739e+01,  1.9350e+01,  3.2556e+01,  1.2531e+00,
        -3.4092e+01, -1.2361e+01, -6.5596e+00, -3.9846e+01, -3.6144e-01,
         8.2413e+01,  2.1191e+01,  1.6397e+01,  3.7534e+02, -4.2903e+01,
        -3.0094e+01, -3.6512e+01,  3.0698e+01, -2.9467e+01,  7.0018e+01,
        -3.3218e+00, -3.1271e+01,  6.8098e+01, -1.7332e+01, -7.7662e+00,
         5.1774e+01, -1.0644e+01,  1.5994e+01,  5.8949e+00,  5.1459e+00,
        -2.1521e+01,  8.8886e+00,  2.3189e+01, -8.8864e+00, -1.4141e+01,
        -7.5629e+00,  1.2571e+01, -1.3532e+01, -2.4706e+01,  4.2030e+01,
        -2.2270e+01,  2.0323e+00, -1.7053e+01,  1.9941e+01,  1.6843e+01,
        -5.1337e+01, -1.6058e+01, -7.4931e+00, -7.0993e+00, -4.5603e+01,
        -1.8836e+00,  4.9463e+01, -2.9742e+01, -3.5303e+01,  1.6317e+01,
         2.7669e+01, -8.8650e+00,  1.1190e+01,  4.4

In [40]:
stop

NameError: name 'stop' is not defined

# Load from Local

In [5]:
from pathlib import Path
from huggingface_hub import snapshot_download
path = Path(snapshot_download(repo_id="EvolutionaryScale/esmc-600m-2024-12"))
print(path)

Fetching 3 files: 100%|██████████| 3/3 [00:56<00:00, 18.75s/it]

/stor/home/lcv454/.cache/huggingface/hub/models--EvolutionaryScale--esmc-600m-2024-12/snapshots/226a1f81631628c20c8e7232d7934b4c6cb90913


In [77]:
model_path = '/stor/home/lcv454/.cache/huggingface/hub/models--EvolutionaryScale--esmc-300m-2024-12/snapshots/15e3c02ec986679ecddd449c02c7186cf554b985/data/weights/esmc_300m_2024_12_v0.pth'
#model_path = '/stor/home/lcv454/.cache/huggingface/hub/models--EvolutionaryScale--esmc-600m-2024-12/snapshots/226a1f81631628c20c8e7232d7934b4c6cb90913/data/weights/esmc_600m_2024_12_v0.pth'

In [75]:
import torch
from esm.models.esmc import ESMC
from esm.tokenization import get_esmc_model_tokenizers


def ESMC_300M_202412(model_path: str, device: torch.device | str = "cpu"):
    with torch.device(device):
        model = ESMC(
            d_model=960, n_heads=15, n_layers=30, tokenizer=get_esmc_model_tokenizers()
        ).eval()

    state_dict = torch.load(model_path, map_location=device,)
    model.load_state_dict(state_dict)
    # Convert model parameters to bfloat16
    #model = model.to(torch.bfloat16)
    model = model.to(torch.float32)

    return model


def ESMC_600M_202412(model_path: str, device: torch.device | str = "cpu"):
    with torch.device(device):
        model = ESMC(
            d_model=1152, n_heads=18, n_layers=36, tokenizer=get_esmc_model_tokenizers()
        ).eval()
    
    state_dict = torch.load(model_path, map_location=device,)
    model.load_state_dict(state_dict)
    # Convert model parameters to float32
    model = model.to(torch.float32)

    return model

In [78]:
custom_model = ESMC_300M_202412(model_path, "cuda")
#custom_model = ESMC_600M_202412(model_path, "cuda")
print("Device", custom_model.device)
custom_model

/tmp/ipykernel_1638957/3117673507.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path, map_location=device,)


Device cuda:0


ESMC(
  (embed): Embedding(64, 960)
  (transformer): TransformerStack(
    (blocks): ModuleList(
      (0-29): 30 x UnifiedTransformerBlock(
        (attn): MultiHeadAttention(
          (layernorm_qkv): Sequential(
            (0): LayerNorm((960,), eps=1e-05, elementwise_affine=True)
            (1): Linear(in_features=960, out_features=2880, bias=False)
          )
          (out_proj): Linear(in_features=960, out_features=960, bias=False)
          (q_ln): LayerNorm((960,), eps=1e-05, elementwise_affine=True)
          (k_ln): LayerNorm((960,), eps=1e-05, elementwise_affine=True)
          (rotary): RotaryEmbedding()
        )
        (ffn): Sequential(
          (0): LayerNorm((960,), eps=1e-05, elementwise_affine=True)
          (1): Linear(in_features=960, out_features=5120, bias=False)
          (2): SwiGLU()
          (3): Linear(in_features=2560, out_features=960, bias=False)
        )
      )
    )
    (norm): LayerNorm((960,), eps=1e-05, elementwise_affine=True)
  )
  (sequ

In [86]:
sequences = ["AA", "AAA", "AAAAA"]
logits, embeddings, hiddens = extract(custom_model, sequences)

In [91]:
embeddings[2, 1:-1, :].detach().cpu().squeeze(0).shape

torch.Size([5, 960])

In [67]:
representations = hiddens[-1]
print(representations.shape)
representations[0][0]

torch.Size([7, 288, 960])


tensor([ 2.6504e+01,  8.5209e+01,  2.7654e+00, -6.1944e+01, -4.8077e+01,
        -4.0412e+01, -5.4907e+01,  1.1671e+02,  6.4442e+00, -5.0825e+01,
        -3.2318e+01, -8.3447e+01,  1.4243e+02,  4.2467e+01, -2.7624e+02,
        -5.6971e+01, -9.8260e+01, -7.2435e+01, -1.2580e+02, -1.2523e+02,
         9.6887e+01, -9.6029e+01,  1.5023e+01,  6.4379e+02,  1.4658e+02,
        -2.5259e+01, -9.3657e+01, -2.7635e+01, -2.2957e+01,  7.2403e+01,
         2.0647e+00, -4.8618e+01,  1.2638e+02,  1.0582e+02, -2.1423e+01,
        -2.2720e+01,  8.5482e+00,  5.0017e+00, -6.5632e+00,  6.9960e+01,
        -3.3360e+01,  8.8610e+00,  6.0461e+01, -3.5431e+01, -3.7951e+01,
        -6.9472e+01,  8.0100e+00,  4.7792e+00,  1.4238e+01,  2.2161e+01,
         4.2488e+00,  2.4308e+01,  1.0211e+01,  2.5086e+02,  3.2126e+01,
         6.0596e+01,  8.6257e+01,  1.8820e+01, -1.1925e+01,  2.4125e+01,
        -1.8647e+00, -3.6488e+01, -3.9737e+00,  1.7308e+01,  1.0654e+02,
         1.3744e+01,  6.1214e+01,  2.8856e+01,  8.5

In [ ]:
seq_length = 2
representations[0, 1:seq_length+1, :].mean(dim=0)

In [ ]:
# multiple sequences
sequences = ["AA", "AAA", "AAAAA"]
logits, embeddings, hiddens = extract(custom_model, sequences)

In [ ]:
representations = hiddens[29]
print(representations[0].shape)
representations[0][0] # first sequence, BOS token 

torch.Size([7, 960])


tensor([ 1.4949e+01, -1.3112e+01,  3.2395e+01, -2.6281e+01,  8.4839e+00,
         5.2685e+01,  1.9758e+01,  3.9826e+00,  1.4584e+01, -1.1302e+01,
        -1.1128e+02, -3.2816e+01,  6.6952e+00,  1.2877e+01, -1.7858e+02,
         1.0357e+01,  7.6590e+01, -8.2369e+01, -2.9855e+00, -7.0749e+01,
         7.8882e+01, -1.1498e+02,  6.0685e+01,  5.4017e+02,  9.9163e+01,
        -1.2015e+01, -7.2386e+01, -5.3812e+01, -3.7315e+01, -2.6345e+01,
         3.1113e+01, -2.4019e+01,  3.2537e+01,  2.0047e+01, -8.8353e+01,
        -6.5127e+01, -4.7641e+01,  3.5897e+00, -9.6808e+01,  8.7257e-01,
        -1.8294e+01, -3.1028e+01, -2.9620e+00, -3.5663e+01, -8.8351e+00,
        -4.7426e+00, -7.2698e+00, -3.3175e+01,  1.0246e+01,  3.8167e+01,
        -2.8453e+01,  9.4772e+00,  8.1723e+00,  7.3805e+01, -1.6134e+01,
         3.1316e+01,  8.4398e+01, -7.8322e+01,  4.3770e+01,  8.4215e+01,
        -2.2152e+01, -8.3945e+01, -8.4086e+01,  2.1227e+01, -1.5096e+01,
        -6.8631e-01, -5.0688e+01,  3.8952e+01,  3.5

In [92]:
from Bio import SeqIO

def fasta_data_loader(fasta_file, batch_size, model):
    """
    Data loader for reading a FASTA file and returning batches of sequence IDs, lengths, and tokenized sequences.
    
    Args:
    - fasta_file (str): Path to the FASTA file.
    - batch_size (int): Number of sequences per batch.
    - model (object): Model object with a `_tokenize` method for tokenizing sequences.
    
    Yields:
    - tuple: A batch containing (ids, lengths, tokens).
    """
    ids, lengths, seqs = [], [], []

    for seq in SeqIO.parse(fasta_file, "fasta"):
        ids.append(seq.id)
        lengths.append(len(seq.seq))
        seqs.append(str(seq.seq))
        
        if len(ids) == batch_size:
            tokens = model._tokenize(seqs)
            yield ids, lengths, tokens
            ids, lengths, seqs = [], [], []

    # Yield any remaining sequences if they don't fill the last batch
    if ids:
        tokens = model._tokenize(seqs)
        yield ids, lengths, tokens


In [93]:
from Bio import SeqIO

class FastaDataLoader:
    """
    Data loader for reading a FASTA file and returning batches of sequence IDs, lengths, and tokenized sequences.
    Designed to work with tqdm.
    
    Args:
    - fasta_file (str): Path to the FASTA file.
    - batch_size (int): Number of sequences per batch.
    - model (object): Model object with a `_tokenize` method for tokenizing sequences.
    """
    def __init__(self, fasta_file, batch_size, model):
        self.fasta_file = fasta_file
        self.batch_size = batch_size
        self.model = model
        self.sequences = list(SeqIO.parse(fasta_file, "fasta"))
        self.total = len(self.sequences)

    def __len__(self):
        return (self.total + self.batch_size - 1) // self.batch_size

    def __iter__(self):
        ids, lengths, seqs = [], [], []
        for seq in self.sequences:
            ids.append(seq.id)
            lengths.append(len(seq.seq))
            seqs.append(str(seq.seq))
            
            if len(ids) == self.batch_size:
                tokens = self.model._tokenize(seqs)
                yield ids, lengths, tokens
                ids, lengths, seqs = [], [], []

        # Yield any remaining sequences if they don't fill the last batch
        if ids:
            tokens = self.model._tokenize(seqs)
            yield ids, lengths, tokens


In [94]:
from Bio import SeqIO

class TokenBasedFastaDataLoader:
    """
    Data loader for reading a FASTA file and creating batches based on a token limit.
    
    Args:
    - fasta_file (str): Path to the FASTA file.
    - batch_token_limit (int, optional): Maximum number of tokens per batch. Defaults to 4096.
    - model (object): Model object with a `_tokenize` method for tokenizing sequences.
    """
    def __init__(self, fasta_file, model, batch_token_limit=4096):
        self.fasta_file = fasta_file
        self.batch_token_limit = batch_token_limit
        self.model = model
        self.sequences = list(SeqIO.parse(fasta_file, "fasta"))
        self.total_sequences = len(self.sequences)
        
        # Check for duplicate sequence labels
        sequence_labels = [seq.id for seq in self.sequences]
        assert len(set(sequence_labels)) == len(sequence_labels), "Found duplicate sequence labels"

    def __len__(self):
        # Approximate total number of batches
        total_tokens = sum(len(str(seq.seq)) + 2 for seq in self.sequences)  # +2 for BOS and EOS tokens
        return (total_tokens + self.batch_token_limit - 1) // self.batch_token_limit

    def __iter__(self):
        ids, lengths, seqs = [], [], []
        current_token_count = 0

        for seq in self.sequences:
            seq_length = len(seq.seq)
            token_count = seq_length + 2  # Include BOS and EOS tokens
            if current_token_count + token_count > self.batch_token_limit and ids:
                # Yield current batch if adding the new sequence exceeds the token limit
                tokens = self.model._tokenize(seqs)
                yield ids, lengths, tokens
                ids, lengths, seqs = [], [], []
                current_token_count = 0

            # Add the current sequence to the batch
            ids.append(seq.id)
            lengths.append(seq_length)
            seqs.append(str(seq.seq))
            current_token_count += token_count

        # Yield any remaining sequences
        if ids:
            tokens = self.model._tokenize(seqs)
            yield ids, lengths, tokens


In [ ]:
from tqdm import tqdm
import pandas as pd

def extract_mean_representations(model, fasta_file):
    mean_representations = {}

    #data_loader = FastaDataLoader(fasta_file, batch_size=batch_size, model=model)
    data_loader = TokenBasedFastaDataLoader(fasta_file, model=model)
    
    for batch_ids, batch_lengths, batch_tokens in tqdm(data_loader, desc="Processing batches", leave=False):
        output = model(batch_tokens)
        logits, embeddings, hiddens = (
            output.sequence_logits,
            output.embeddings,
            output.hidden_states,
        )

        for i, ID in enumerate(batch_ids):
            # Extract the last hidden states for the sequence
            representations =  embeddings[i, 1:batch_lengths[i]+1, :].detach().to('cpu') 
            # compute mean representation of the sequence
            mean_representations[ID] = (representations.mean(dim=0))

    return mean_representations

In [99]:
fasta_file = "test/test.fasta"
embed = extract_mean_representations(model, fasta_file)
embed 

{'BLAT_Ten_H24P': tensor([ 6.9843e-03,  8.0278e-03,  4.7115e-03, -1.0907e-02,  2.3243e-03,
         -1.1672e-02, -1.5283e-03,  6.8056e-03, -1.1118e-02,  4.8469e-03,
          2.3747e-02, -6.6120e-03,  6.6625e-03,  1.0214e-02,  5.3346e-04,
         -1.0483e-02, -3.9338e-03, -1.5267e-03, -1.2002e-02,  1.4175e-04,
          3.4053e-02,  6.8003e-03,  6.0194e-03,  3.6529e-01, -1.4280e-02,
         -8.3578e-03, -9.8729e-03,  1.0282e-02, -8.7031e-03,  2.2794e-02,
         -2.3564e-04, -9.7974e-03,  2.0336e-02, -4.2142e-03, -1.3885e-03,
          1.6778e-02, -2.0712e-03,  5.1504e-03,  1.7224e-03,  2.3181e-03,
         -5.6378e-03,  3.5025e-03,  7.1796e-03, -1.9683e-03, -3.5878e-03,
         -1.7950e-03,  4.4818e-03, -2.7885e-03, -6.0271e-03,  1.3499e-02,
         -5.9979e-03,  1.8381e-03, -4.1935e-03,  7.8717e-03,  5.4776e-03,
         -1.5615e-02, -3.5292e-03, -2.0912e-03, -1.1504e-03, -1.3111e-02,
         -2.3605e-04,  1.5062e-02, -7.3223e-03, -8.9081e-03,  4.7992e-03,
          8.1069e-03,

In [100]:
pd.DataFrame(embed).T

,0,1,2,3,4,5,6,7,8,9,...,950,951,952,953,954,955,956,957,958,959
BLAT_Ten_H24P,0.006984,0.008028,0.004712,-0.010907,0.002324,-0.011672,-0.001528,0.006806,-0.011118,0.004847,...,0.003584,-0.013235,0.006926,-0.007619,-0.004341,0.001519,0.008878,0.004381,-0.010453,-0.011566
BLAT_Ten_H24R,0.007205,0.005590,0.005324,-0.009390,0.003593,-0.010016,-0.000764,0.006752,-0.008867,0.004351,...,0.002126,-0.015569,0.006312,-0.009730,-0.006598,0.002194,0.008698,0.002053,-0.012063,-0.012290
BLAT_Ten_H24Y,0.008238,0.004314,0.003166,-0.007733,0.006063,-0.008805,-0.001099,0.007184,-0.007740,0.004009,...,0.001574,-0.017620,0.007568,-0.010811,-0.005666,0.003513,0.007035,0.001922,-0.013512,-0.011414
BLAT_Ten_H24L,0.008274,0.002855,0.003172,-0.007030,0.006393,-0.008077,-0.000860,0.007102,-0.007270,0.003899,...,0.001641,-0.017725,0.007876,-0.011741,-0.006275,0.003508,0.006620,0.001637,-0.014132,-0.011115


In [ ]:
stop

NameError: name 'stop' is not defined

# Using ESM C 6B via Forge API

In [1]:
with open('tokens/ESM3_token.txt', 'r') as f:
    my_token = f.readlines()[0].strip()

In [4]:
from esm.sdk.forge import ESM3ForgeInferenceClient
from esm.sdk.api import ESMProtein, LogitsConfig

# Apply for forge access and get an access token
forge_client = ESM3ForgeInferenceClient(model="esmc-6b-2024-12", url="https://forge.evolutionaryscale.ai", token=my_token)

protein = ESMProtein(sequence="AAAAA")

protein_tensor = forge_client.encode(protein)

logits_output = forge_client.logits(protein_tensor, LogitsConfig(sequence=True, return_embeddings=True))

print(logits_output.embeddings.shape)

Retrying... Attempt 1 after 1.0s due to: (429, 'Failure in encode: {"status":"error","message":"You have exceeded your free daily credit limit of 1000 tokens."}')
Retrying... Attempt 2 after 2.0s due to: (429, 'Failure in encode: {"status":"error","message":"You have exceeded your free daily credit limit of 1000 tokens."}')
Retrying... Attempt 3 after 4.0s due to: (429, 'Failure in encode: {"status":"error","message":"You have exceeded your free daily credit limit of 1000 tokens."}')
Retrying... Attempt 4 after 8.0s due to: (429, 'Failure in encode: {"status":"error","message":"You have exceeded your free daily credit limit of 1000 tokens."}')


RetryError: RetryError[<Future at 0x7f07acb65f00 state=finished returned ESMProteinError>]

In [ ]:
import time
from datetime import datetime, timedelta

def send_job():
    # Apply for forge access and get an access token
    forge_client = ESM3ForgeInferenceClient(model="esmc-6b-2024-12", url="https://forge.evolutionaryscale.ai", token=my_token)

    protein = ESMProtein(sequence="AAAAA")

    protein_tensor = forge_client.encode(protein)

    logits_output = forge_client.logits(protein_tensor, LogitsConfig(sequence=True, return_embeddings=True))

    print(logits_output.embeddings.shape)


def calculate_sleep_time_until_6pm():
    now = datetime.now()
    six_pm = now.replace(hour=13, minute=25, second=0, microsecond=0)
    if now > six_pm:  # If it's already past 6 PM, set for the next day
        six_pm += timedelta(days=1)
    return (six_pm - now).total_seconds()



max_retries = 5 
retry_count = 0

while retry_count < max_retries:
    try:
        send_job()
        break  # Exit loop if successful
    except Exception as e:
        if "RetryError" in str(e):
            print(f"RetryError encountered. Attempt {retry_count + 1} of {max_retries}.")
            sleep_seconds = calculate_sleep_time_until_6pm()
            print(f"Sleeping for {int(sleep_seconds)} seconds until 6 PM...")
            time.sleep(sleep_seconds)  # Sleep until 6 PM
            retry_count += 1
        else:
            print(f"An unexpected error occurred: {e}")
            break  # Exit loop for non-retryable errors
else:
    print("Max retries reached. Exiting.")

In [19]:
from esm.sdk.api import SamplingConfig

# Create a SamplingConfig object
sampling_config = SamplingConfig(
    return_mean_embedding=True,  # Specify whether to return the mean embedding
    return_per_residue_embeddings=False  # Specify whether to return per-residue embeddings
)

# Call the method with both arguments
result = forge_client.forward_and_sample(protein_tensor, SamplingConfig(return_per_residue_embeddings=True,))
result

ESMProteinError(error_code=422, error_msg='Failure in forward_and_sample: {"status":"error","message":"422: Model esmc-6b-2024-12 is not supported for endpoint forward_and_sample"}')

In [3]:
import torch
import time
from Bio import SeqIO
from esm.sdk.forge import ESM3ForgeInferenceClient
from esm.sdk.api import ESMProtein, LogitsConfig
from tqdm import tqdm

# Load the token
with open('tokens/ESM3_token.txt', 'r') as f:
    my_token = f.readlines()[0].strip()

# Load the ESM-6B model
# Apply for forge access and get an access token
print("Loading the ESM-6B model")
forge_client = ESM3ForgeInferenceClient(model="esmc-6b-2024-12", url="https://forge.evolutionaryscale.ai", token=my_token)

datasets = ['POLG_HCVJF_Sun2014', 'BRCA1_HUMAN_RING', 'BRCA1_HUMAN_BRCT', 'UBE4B_MOUSE_Klevit2013_singles']
for dataset in datasets:
    print(f'Reading the input file: {dataset}')
    base_dir = 'data/DMS_mut_sequences'
    input_file = f"{base_dir}/{dataset}_muts.fasta"
    output_dir = f"embeddings/DMS_all/esmc_6B/{dataset}"

    total_records = len(list(SeqIO.parse(input_file, "fasta")))
    print(f'Total records: {total_records}')
    records = SeqIO.parse(input_file, "fasta")
    count = 0

    # I can only process 50 files per minute
    start_time = time.time()
    batch_size = 50
    interval = 60


    for seq_record in tqdm(records, total=total_records):
        count += 1
        
        label = seq_record.id
        sequence = str(seq_record.seq)
        print(f'Processing record: {label}')
        print(f'Sequence: {sequence}')
    
        protein = ESMProtein(sequence=sequence)
        protein_tensor = forge_client.encode(protein)
        logits_output = forge_client.logits(protein_tensor, LogitsConfig(sequence=True, return_embeddings=True))
        print(logits_output.embeddings)

        break
    break

Loading the ESM-6B model
Reading the input file: POLG_HCVJF_Sun2014
Total records: 1716


  0%|          | 0/1716 [00:00<?, ?it/s]

Processing record: POLG_HCVJF_D1994C
Sequence: MSTNPKPQRKTKRNTNRRPEDVKFPGGGQIVGGVYLLPRRGPRLGVRTTRKTSERSQPRGRRQPIPKDRRSTGKAWGKPGRPWPLYGNEGLGWAGWLLSPRGSRPSWGPTDPRHRSRNVGKVIDTLTCGFADLMGYIPVVGAPLSGAARAVAHGVRVLEDGVNYATGNLPGFPFSIFLLALLSCITVPVSAAQVKNTSSSYMVTNDCSNDSITWQLEAAVLHVPGCVPCERVGNTSRCWVPVSPNMAVRQPGALTQGLRTHIDMVVMSATFCSALYVGDLCGGVMLAAQVFIVSPQYHWFVQECNCSIYPGTITGHRMAWDMMMNWSPTATMILAYVMRVPEVIIDIVSGAHWGVMFGLAYFSMQGAWAKVIVILLLAAGVDAGTTTVGGAVARSTNVIAGVFSHGPQQNIQLINTNGSWHINRTALNCNDSLNTGFLAALFYTNRFNSSGCPGRLSACRNIEAFRIGWGTLQYEDNVTNPEDMRPYCWHYPPKPCGVVPARSVCGPVYCFTPSPVVVGTTDRRGVPTYTWGENETDVFLLNSTRPPQGSWFGCTWMNSTGFTKTCGAPPCRTRADFNASTDLLCPTDCFRKHPDATYIKCGSGPWLTPKCLVHYPYRLWHYPCTVNFTIFKIRMYVGGVEHRLTAACNFTRGDRCDLEDRDRSQLSPLLHSTTEWAILPCTYSDLPALSTGLLHLHQNIVDVQYMYGLSPAITKYVVRWEWVVLLFLLLADARVCACLWMLILLGQAEAALEKLVVLHAASAANCHGLLYFAIFFVAAWHIRGRVVPLTTYCLTGLWPFCLLLMALPRQAYAYDAPVHGQIGVGLLILITLFTLTPGYKTLLGQCLWWLCYLLTLGEAMIQEWVPPMQVRGGRDGIAWAVTIFCPGVVFDITKWLLALLGPAYLLRAALTHVPYFVRAHALIRVCALVKQLAGGRYVQVALLALGRWTGTYI

  0%|          | 0/1716 [00:00<?, ?it/s]


AttributeError: 'ESMProteinError' object has no attribute 'embeddings'

In [5]:
logits_output = forge_client.logits(protein_tensor, LogitsConfig(sequence=True, return_embeddings=True))
logits_output

ESMProteinError(error_code=422, error_msg='Failure in logits: {"status":"error","message":"Cannot forward input with seq_len=3035, this model only supports seq_len<=2048"}')

# ESM 3

In [18]:
my_token = '5oFzMEeffgpDdaxchfceg4'

In [ ]:
from esm.sdk import client
from esm.sdk.api import (
    ESM3InferenceClient,
    ESMProtein,
    ESMProteinError,
    ESMProteinTensor,
    GenerationConfig,
    LogitsConfig,
    LogitsOutput,
    SamplingConfig,
    SamplingTrackConfig,
)

model = client(
    model="esm3-medium-2024-03",
    url="https://forge.evolutionaryscale.ai",
    token=my_token,
)

In [ ]:
aa_seq = 'AAAAA' 
protein = ESMProtein(sequence=str(aa_seq))
protein_tensor = model.encode(protein)
output = model.forward_and_sample(protein_tensor, SamplingConfig(return_per_residue_embeddings=True,return_mean_embedding=True,))
output.mean_embedding

ForwardAndSampleOutput(logits=None, embeddings=None, residue_annotation_logits=None, protein_tensor=ESMProteinTensor(sequence=tensor([0, 5, 5, 5, 5, 5, 2]), structure=tensor([4098, 4096, 4096, 4096, 4096, 4096, 4097]), secondary_structure=tensor([0, 0, 0, 0, 0, 0, 0]), sasa=tensor([0, 0, 0, 0, 0, 0, 0]), function=tensor([[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]]), residue_annotations=None, coordinates=None, potential_sequence_of_concern=False), entropy=ForwardTrackData(sequence=None, structure=None, secondary_structure=None, sasa=None, function=None), prob=ForwardTrackData(sequence=None, structure=None, secondary_structure=None, sasa=None, function=None), logprob=ForwardTrackData(sequence=None, structure=None, secondary_structure=None, sasa=None, function=None), top_prob=None, topk_logprob=ForwardTrac

In [25]:
len(output.mean_embedding)

2560

In [27]:
import numpy as np
np.array(output.per_residue_embedding)[1:-1].shape

(5, 2560)

In [28]:
protein

ESMProtein(sequence='AAAAA', secondary_structure=None, sasa=None, function_annotations=None, coordinates=None, plddt=None, ptm=None, potential_sequence_of_concern=False)

In [ ]:
protein_tensor

ESMProteinTensor(sequence=tensor([0, 5, 5, 5, 5, 5, 2]), structure=None, secondary_structure=None, sasa=None, function=None, residue_annotations=None, coordinates=None, potential_sequence_of_concern=False)

In [ ]:
from huggingface_hub import login
from esm.models.esm3 import ESM3
from esm.sdk.api import ESM3InferenceClient, ESMProtein, GenerationConfig

# Will instruct you how to get an API key from huggingface hub, make one with "Read" permission.
#login()

# This will download the model weights and instantiate the model on your machine.
#ESM3InferenceClient = ESM3.from_pretrained("esm3-open")#.to("cuda") # or "cpu"
ESM3InferenceClient = ESM3.from_pretrained("esm3_sm_open_v1")
ESM3InferenceClient

Fetching 22 files: 100%|██████████| 22/22 [01:25<00:00,  3.90s/it]
/stor/work/Wilke/luiz/ESM3/venv/lib/python3.10/site-packages/esm/pretrained.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this 

ESM3(
  (encoder): EncodeInputs(
    (sequence_embed): Embedding(64, 1536)
    (plddt_projection): Linear(in_features=16, out_features=1536, bias=True)
    (structure_per_res_plddt_projection): Linear(in_features=16, out_features=1536, bias=True)
    (structure_tokens_embed): Embedding(4101, 1536)
    (ss8_embed): Embedding(11, 1536)
    (sasa_embed): Embedding(19, 1536)
    (function_embed): ModuleList(
      (0-7): 8 x Embedding(260, 192, padding_idx=0)
    )
    (residue_embed): EmbeddingBag(1478, 1536, mode='sum', padding_idx=0)
  )
  (transformer): TransformerStack(
    (blocks): ModuleList(
      (0): UnifiedTransformerBlock(
        (attn): MultiHeadAttention(
          (layernorm_qkv): Sequential(
            (0): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
            (1): Linear(in_features=1536, out_features=4608, bias=False)
          )
          (out_proj): Linear(in_features=1536, out_features=1536, bias=False)
          (q_ln): LayerNorm((1536,), eps=1e-05, el

In [ ]:
# Generate a completion for a partial Carbonic Anhydrase (2vvb)
prompt = "___________________________________________________DQATSLRILNNGHAFNVEFDDSQDKAVLKGGPLDGTYRLIQFHFHWGSLDGQGSEHTVDKKKYAAELHLVHWNTKYGDFGKAVQQPDGLAVLGIFLKVGSAKPGLQKVVDVLDSIKTKGKSADFTNFDPRGLLPESLDYWTYPGSLTTPP___________________________________________________________"
protein = ESMProtein(sequence=prompt)
# Generate the sequence, then the structure. This will iteratively unmask the sequence track.
protein = ESM3InferenceClient.generate(protein, GenerationConfig(track="sequence", num_steps=8, temperature=0.7))
protein

100%|██████████| 8/8 [00:00<00:00, 10.99it/s]
/stor/work/Wilke/luiz/ESM3/venv/lib/python3.10/site-packages/esm/pretrained.py:46: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.


ESMProtein(sequence='MLMVLGLLAPGAGKGGAPQCCNGKIGPQQSPDLHVGKLKKKELKPLQFSGYDQATSLRILNNGHAFNVEFDDSQDKAVLKGGPLDGTYRLIQFHFHWGSLDGQGSEHTVDKKKYAAELHLVHWNTKYGDFGKAVQQPDGLAVLGIFLKVGSAKPGLQKVVDVLDSIKTKGKSADFTNFDPRGLLPESLDYWTYPGSLTTPPCSESVTWIVLKDPIEISSDQLKKFRSLLSVKDGEPSLVNTFRPPQQDGVRLVTLFKAAQ', secondary_structure=None, sasa=None, function_annotations=None, coordinates=None, plddt=None, ptm=None, potential_sequence_of_concern=False)